### CourseDataWarehouse

In [1]:
%load_ext sql
import pandas as pd
import sqlite3

In [2]:
%sql sqlite:///CourseDataWarehouse.db
conn = sqlite3.connect('CourseDataWarehouse.db')

In [3]:
%%sql

-- Drop the tables incase they already exist 

DROP TABLE IF EXISTS Courses;
DROP TABLE IF EXISTS Course_Meeting;
DROP TABLE IF EXISTS Catalog;
DROP TABLE IF EXISTS Program;
DROP TABLE IF EXISTS Fees;


DROP TABLE IF EXISTS IMPORT_COURSE_OFFERINGS;
DROP TABLE IF EXISTS IMPORT_COURSE_MEETINGS;
DROP TABLE IF EXISTS IMPORT_CATALOG_COURSES;


 * sqlite:///CourseDataWarehouse.db
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.


[]

##### Create ERD tables - We are recreating the tables from the database to begin with

In [4]:
%%sql

CREATE TABLE Courses (
    CRN SMALLINT,
    Term VARCHAR(20),
    Section VARCHAR(10),
    Cat_ID VARCHAR(10),
    Faculty_Name VARCHAR(10),
    Credits TINYINT,
    Capacity SMALLINT,
    Actual SMALLINT,
    Remaining SMALLINT,
    PRIMARY KEY (CRN, Term),
    FOREIGN KEY(Cat_ID) REFERENCES Catalog(Cat_ID)
);

 * sqlite:///CourseDataWarehouse.db
Done.


[]

In [5]:
%%sql

CREATE TABLE Course_Meeting (
    MID INTEGER PRIMARY KEY,
    CRN SMALLINT,
    Term VARCHAR(20),
    Room TEXT,
    Day TEXT,
    Start_Time TIME,
    End_Time TIME,  
    FOREIGN KEY(CRN,Term) REFERENCES Courses(CRN,Term)
);

 * sqlite:///CourseDataWarehouse.db
Done.


[]

In [6]:

%%sql

CREATE TABLE Catalog (
    Cat_ID VARCHAR(10) PRIMARY KEY,
    Prg_Code VARCHAR(5),
    Title VARCHAR(150),
    Prerequisites VARCHAR(10),
    Corequisites VARCHAR(10),
    FOREIGN KEY(Prg_Code) REFERENCES Program(Prg_Code)
);

 * sqlite:///CourseDataWarehouse.db
Done.


[]

In [7]:
%%sql

CREATE TABLE Program (
    Prg_Code VARCHAR(10) PRIMARY KEY,
    Program_Name VARCHAR(150)
);

 * sqlite:///CourseDataWarehouse.db
Done.


[]

In [8]:
%%sql

CREATE TABLE Fees (
    Cat_ID VARCHAR(10),
    Academic_Year VARCHAR(20),
    Fee TEXT,
    PRIMARY KEY (Cat_ID, Academic_Year)
    FOREIGN KEY(Cat_ID) REFERENCES Catalog(Cat_ID)
);

 * sqlite:///CourseDataWarehouse.db
Done.


[]

In [9]:
#Catalog Data - Loading all the catalogs csv files into a SQL table called IMPORT_CATALOG_COURSES

catalog_years = ['2017_2018', '2018_2019']

for cat_year in catalog_years:
    filepath = '../SourceData/Catalogs/CourseCatalog'+cat_year+'.csv'
    data = pd.read_csv(filepath)
    data['cat_year'] = cat_year
    data.to_sql('IMPORT_CATALOG_COURSES',conn,if_exists='append',index=False)

In [10]:
#Course Offering and Course Meeting Data - Loading all the Course Offering and Course Meeting csv files into a SQL table called IMPORT_CATALOG_COURSES

terms = ['Fall2014','Fall2015','Fall2016','Fall2017','Fall2018',
         'Spring2015','Spring2016','Spring2017','Spring2018','Spring2019',
        'SpringBreak2017',
         'Summer2015','Summer2016','Summer2017','Summer2018',
         'Winter2015','Winter2016','Winter2017','Winter2018']

for term in terms:
    filepath = '../SourceData/'+term+'/courses.csv'
    data = pd.read_csv(filepath)
    data.to_sql('IMPORT_COURSE_OFFERINGS',conn,if_exists='append',index=False) 
    
    filepath = '../SourceData/'+term+'/course_meetings.csv'
    data = pd.read_csv(filepath)
    data.to_sql('IMPORT_COURSE_MEETINGS',conn,if_exists='append',index=False)



##### Insert into ERD tables


-- Insert the data from IMPORT_CATALOG_COURSES into the Program table --

In [11]:
%%sql

INSERT INTO Program (Prg_Code, Program_Name)
SELECT DISTINCT program_code, program_name 
FROM IMPORT_CATALOG_COURSES;

 * sqlite:///CourseDataWarehouse.db
83 rows affected.


[]

-- Insert the data from IMPORT_CATALOG_COURSES into the Catalog table and linking to Program table --

In [12]:
%%sql

INSERT INTO Catalog (Cat_ID, Prg_Code, Title, Prerequisites, Corequisites)
SELECT DISTINCT catalog_id, program_code, course_title,prereqs,coreqs
FROM IMPORT_CATALOG_COURSES 
    JOIN Program ON (Prg_Code = program_code);

 * sqlite:///CourseDataWarehouse.db
2220 rows affected.


[]

-- Insert the data from IMPORT_COURSE_MEETINGS into the Course_Meeting table 

In [13]:
%%sql

INSERT INTO Course_Meeting (CRN, Term, Room, Day, Start_Time, End_Time)
SELECT DISTINCT crn, term, location, day, start, end
FROM IMPORT_COURSE_MEETINGS;

 * sqlite:///CourseDataWarehouse.db
311141 rows affected.


[]

-- Insert the data from IMPORT_COURSE_OFFERINGS into the Courses table and linking the Course Meeting and Catalog tables

In [14]:
%%sql

INSERT INTO Courses (CRN, Term, Section, Cat_ID, Faculty_Name, Credits, Capacity, Actual, Remaining)
SELECT DISTINCT IMPORT_COURSE_OFFERINGS.crn, IMPORT_COURSE_OFFERINGS.term, section, catalog_id, primary_instructor, credits, cap, act, rem
FROM IMPORT_COURSE_OFFERINGS
    LEFT JOIN Course_Meeting ON Course_Meeting.CRN = IMPORT_COURSE_OFFERINGS.crn AND Course_Meeting.Term = IMPORT_COURSE_OFFERINGS.term
    LEFT JOIN Catalog ON Catalog.Cat_ID = IMPORT_COURSE_OFFERINGS.catalog_id;

 * sqlite:///CourseDataWarehouse.db
15937 rows affected.


[]

-- Insert the data from IMPORT_COURSE_COURSES into the Fees table and linking the Catalog Table

In [15]:
%%sql

INSERT INTO Fees (Cat_ID, Academic_Year, Fee )
SELECT DISTINCT catalog_id, cat_year, fees
FROM IMPORT_CATALOG_COURSES
    LEFT JOIN Catalog ON Catalog.Cat_ID = IMPORT_CATALOG_COURSES.catalog_id;   

 * sqlite:///CourseDataWarehouse.db
4440 rows affected.


[]

**Drop Datawarehouse Tables in case they already exist**

In [16]:
%%sql

DROP TABLE IF EXISTS Location;
DROP TABLE IF EXISTS Course_Facts;
DROP TABLE IF EXISTS Meeting;
DROP TABLE IF EXISTS Fees1;
DROP TABLE IF EXISTS Program1;


 * sqlite:///CourseDataWarehouse.db
Done.
Done.
Done.
Done.
Done.


[]

## Create New Datawarehouse Tables

In [17]:
%%sql

CREATE TABLE Location (
    LID INTEGER PRIMARY KEY,
    Room TEXT
);

 * sqlite:///CourseDataWarehouse.db
Done.


[]

-- Insert the data from Course_Meeting into the Location table

In [18]:
%%sql

INSERT INTO Location (Room)
SELECT DISTINCT Room 
FROM Course_Meeting;

 * sqlite:///CourseDataWarehouse.db
207 rows affected.


[]

In [19]:
%%sql

CREATE TABLE Program1 (
    PID INTEGER PRIMARY KEY,
    Prg_Code VARCHAR(10),
    Program_Name VARCHAR(150)
);

 * sqlite:///CourseDataWarehouse.db
Done.


[]

-- Insert the data from Import_Catalog_Courses into the Program1 table

In [20]:
%%sql

INSERT INTO Program1 (Prg_Code, Program_Name)
SELECT DISTINCT Prg_Code, Program_name 
FROM Program;

 * sqlite:///CourseDataWarehouse.db
83 rows affected.


[]

In [21]:
%%sql

CREATE TABLE Meeting (
    MID INTEGER PRIMARY KEY,
    Cat_ID TEXT,
    Day TEXT,
    Start_Time TIME,
    End_Time TIME  
);

 * sqlite:///CourseDataWarehouse.db
Done.


[]

-- Insert the data from Import_Catalog_Courses into the Meeting table

In [22]:
%%sql

INSERT INTO Meeting (Day, Start_Time, End_Time)
SELECT DISTINCT Day, Start_Time, End_Time
FROM Course_Meeting


 * sqlite:///CourseDataWarehouse.db
42551 rows affected.


[]

In [23]:
%%sql

SELECT *
FROM Course_Meeting
LIMIT 25;


 * sqlite:///CourseDataWarehouse.db
Done.


MID,CRN,Term,Room,Day,Start_Time,End_Time
1,70384,Fall2014,DSB 105,T,2014-09-02T08:00:00,2014-09-02T09:15:00
2,70384,Fall2014,DSB 105,F,2014-09-05T08:00:00,2014-09-05T09:15:00
3,70384,Fall2014,DSB 105,T,2014-09-09T08:00:00,2014-09-09T09:15:00
4,70384,Fall2014,DSB 105,F,2014-09-12T08:00:00,2014-09-12T09:15:00
5,70384,Fall2014,DSB 105,T,2014-09-16T08:00:00,2014-09-16T09:15:00
6,70384,Fall2014,DSB 105,F,2014-09-19T08:00:00,2014-09-19T09:15:00
7,70384,Fall2014,DSB 105,T,2014-09-23T08:00:00,2014-09-23T09:15:00
8,70384,Fall2014,DSB 105,F,2014-09-26T08:00:00,2014-09-26T09:15:00
9,70384,Fall2014,DSB 105,T,2014-09-30T08:00:00,2014-09-30T09:15:00
10,70384,Fall2014,DSB 105,F,2014-10-03T08:00:00,2014-10-03T09:15:00


In [24]:
%%sql

CREATE TABLE Fees1 (
    Fee_ID INTEGER PRIMARY KEY,
    Cat_ID VARCHAR(10),
    Academic_Year VARCHAR(20),
    Amount VARCHAR(50)
);

 * sqlite:///CourseDataWarehouse.db
Done.


[]

-- Insert the data from Import_Catalog_Courses into the Fees table

In [25]:
%%sql

INSERT INTO Fees1 (Cat_ID, Academic_Year, Amount )
SELECT Cat_ID, Academic_Year, fee 
FROM Fees
WHERE fee NOT NULL
;


 * sqlite:///CourseDataWarehouse.db
250 rows affected.


[]

In [26]:
%%sql

SELECT ROWID, *
FROM Fees1
LIMIT 25;

 * sqlite:///CourseDataWarehouse.db
Done.


Fee_ID,Fee_ID_1,Cat_ID,Academic_Year,Amount
1,1,BI 0107,2017_2018,$55 Science Lab Fee
2,2,BI 0108,2017_2018,$55 Science Lab Fee
3,3,SA 0011,2017_2018,$55 Materials Fee
4,4,BI 0151,2017_2018,$55 Science Lab Fee
5,5,SA 0012,2017_2018,$55 Materials Fee
6,6,SA 0013,2017_2018,$55 Materials Fee
7,7,SA 0014,2017_2018,$55 Materials Fee
8,8,SA 0015,2017_2018,$55 Materials Fee
9,9,SA 0016,2017_2018,$55 Materials Fee
10,10,BI 0170,2017_2018,$55 Science Lab Fee


In [27]:
%%sql

CREATE TABLE Course_Facts (
    CRN SMALLINT,
    Term VARCHAR(20),
    Faculty_Name VARCHAR(10),
    Title VARCHAR(100),
    Section VARCHAR(10),
    Cat_ID VARCHAR(10),
    LID VARCHAR(10),
    MID VARCHAR(10),
    PID VARCHAR(10),
    FEE_ID VARCHAR(10),
    Capacity SMALLINT,
    Actual SMALLINT,
    Remaining SMALLINT
);

 * sqlite:///CourseDataWarehouse.db
Done.


[]

-- Insert the data from Courses into the Course_Facts table and linking the Foreign keys using the database tables

In [28]:
%%sql
INSERT INTO Course_Facts (CRN, Term, Faculty_Name,Title, Cat_ID,PID, Section,Capacity,Actual,Remaining, LID, MID, Fee_Id)
SELECT Courses.CRN, Courses.Term, Faculty_Name,Catalog.Title, Courses.Cat_ID, Program1.PID,  Section, Capacity, Actual, Remaining, Location.LID, Meeting.MID, Fees1.Fee_ID
FROM Courses
    LEFT JOIN Catalog USING (Cat_ID)
    LEFT JOIN Program1 USING (Prg_Code)
    LEFT JOIN Course_Meeting ON Course_Meeting.CRN = Courses.CRN AND Course_Meeting.Term = Courses.Term
    LEFT JOIN Location ON Course_Meeting.Room = Location.Room
    LEFT JOIN Meeting ON Course_Meeting.Start_Time = Meeting.Start_Time
    LEFT JOIN Fees1 on Fees1.Cat_ID = Catalog.Cat_ID;
    
    

 * sqlite:///CourseDataWarehouse.db
1228773 rows affected.


[]

In [29]:
%%sql
SELECT *
FROM Course_Facts
GROUP BY Title, Section, Term
ORDER BY CRN
LIMIT 50
;

 * sqlite:///CourseDataWarehouse.db
Done.


CRN,Term,Faculty_Name,Title,Section,Cat_ID,LID,MID,PID,FEE_ID,Capacity,Actual,Remaining
10893,Winter2015,Thomas B. Cunningham,Ecology and Society,01,BI 0075,None,None,10,None,20,22,-2
10894,Winter2015,David Gudelunas,Human Communication Theories,01,CO 0100,None,None,17,None,20,14,6
10895,Winter2015,Qin Zhang,Intercultural Communication,01,CO 0240,None,None,17,None,20,11,9
10896,Winter2015,Margaret A. Wills,Family Communication,A,CO 0246,64,42399,17,None,15,12,3
10897,Winter2015,David Gudelunas,None,A,CO 0398,None,None,None,None,0,1,-1
10898,Winter2015,William F. Vasquez Mazariegos,Introduction to Microeconomics,01,EC 0011,None,None,20,None,20,9,11
10900,Winter2015,Gita Rajan,Introduction to Literary and Cultural Studies,01,EN 0101,None,None,26,None,20,20,0
10901,Winter2015,Maeve L. Lucey,Business Writing,01,ENW 0332,None,None,27,None,20,8,12
10902,Winter2015,Robert James Parisi,Introduction to Probability and Statistics,01,MA 0017,None,None,52,None,20,16,4
10903,Winter2015,Brian Q. Torff,History and Development of Rock,A,MU 0102,98,42399,59,None,15,15,0


**Drop Import Tables and Database Tables**

In [30]:
%%sql
DROP TABLE IF EXISTS Courses;
DROP TABLE IF EXISTS Course_Meeting;
DROP TABLE IF EXISTS Catalog;
DROP TABLE IF EXISTS Program;
DROP TABLE IF EXISTS Fees;


DROP TABLE IF EXISTS IMPORT_COURSE_OFFERINGS;
DROP TABLE IF EXISTS IMPORT_COURSE_MEETINGS;
DROP TABLE IF EXISTS IMPORT_CATALOG_COURSES;


 * sqlite:///CourseDataWarehouse.db
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.


[]

## Drop ALL Tables ONLY RUN BEFORE SYNCING WITH GITHUB

In [31]:
%%sql
DROP TABLE IF EXISTS Course_Facts;
DROP TABLE IF EXISTS Meeting;
DROP TABLE IF EXISTS Fees1;
DROP TABLE IF EXISTS Program1;
DROP TABLE IF EXISTS Location;

 * sqlite:///CourseDataWarehouse.db
Done.
Done.
Done.
Done.
Done.


[]

##### Vacuum the database to reduce the size

In [32]:
%%sql
Vacuum;

 * sqlite:///CourseDataWarehouse.db
Done.


[]